Import the modules, create a database connection.

In [ ]:
#import the receipes to the database from xml file

import psycopg2
import xml.etree.ElementTree as ET

#connect to the database
username = 'wsuser'
password = 'Solliance123'
host = 'pgsqldevilkdittaflex16.postgres.database.azure.com'
port = '5432'
dbname = 'ailabs'

dbconn = psycopg2.connect(host=host, user=username, password=password,
    port=port, database=dbname , connect_timeout=10)
dbconn.set_session(autocommit=True)


Create the database tables.

In [ ]:
#create the tables

cur = dbconn.cursor()
cur.execute('DROP TABLE IF EXISTS recipes')
cur.execute('DROP TABLE IF EXISTS ingredients')
cur.execute('DROP TABLE IF EXISTS recipe_ingredients')

cur.execute("""CREATE TABLE public.ingredients (
    id integer NOT NULL,
    name text
);
""")

cur.execute("""CREATE TABLE public.recipe_ingredients (
    id integer NOT NULL,
    recipe_id integer,
    ingredient_id integer,
    quantity text,
    unit_key integer
);
""")

cur.execute("""
        CREATE TABLE public.recipes (
        id integer NOT NULL,
        name text,
        description text,
        category text,
        embedding public.vector(1536),
        compacted_embedding public.vector(50),
        super_compacted_embedding public.vector(10)
    );
""")

Import the CSV data.

In [ ]:
import pandas as pd
#load the table into dataframe
def export_data():
    df = pd.read_sql_query('SELECT * FROM recipes',con=dbconn)
    #output to csv
    df.to_csv('recipes.csv', index=False)

    df = pd.read_sql_query('SELECT * FROM recipe_ingredients',con=dbconn)
    #output to csv
    df.to_csv('recipe_ingredients.csv', index=False)

    df = pd.read_sql_query('SELECT * FROM ingredients',con=dbconn)
    #output to csv
    df.to_csv('ingredients.csv', index=False)

def import_data():
    df = pd.read_csv('recipes.csv')
    df.to_sql('recipes', con=dbconn, if_exists='append', index=False)

    df = pd.read_csv('recipe_ingredients.csv')
    df.to_sql('recipe_ingredients', con=dbconn, if_exists='append', index=False)

    df = pd.read_csv('ingredients.csv')
    df.to_sql('ingredients', con=dbconn, if_exists='append', index=False)

Perform a search for similar items.

In [ ]:
#find similar items
sql = """
SELECT
        recipe_1.id,
        recipe_1.name,
        recipe_2.id,
        recipe_2.name
FROM (SELECT * FROM recipes WHERE name = 'Dish, turkey, curry') recipe_1,
        recipes AS recipe_2
WHERE recipe_1.id != recipe_2.id
ORDER BY recipe_1.embedding <=> recipe_2.embedding
LIMIT 10;
"""

#execute the query
cur = dbconn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
    print(row)



Perform a search for not similar items.

In [13]:
#find items that are not similar
sql = """
SELECT
        recipe_1.id,
        recipe_1.name,
        recipe_2.id,
        recipe_2.name
FROM (SELECT * FROM recipes WHERE name = 'Corn Dog' LIMIT 1) recipe_1,
        recipes AS recipe_2
WHERE
  recipe_2.description IS NOT NULL
ORDER BY recipe_1.embedding <=> recipe_2.embedding DESC
LIMIT 2;
"""

#execute the query
cur = dbconn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
    print(row)

(165, 'Corn Dog', 538, 'Salad, green, tossed')
(165, 'Corn Dog', 519, 'Salad, chicken, tropical')
